In [0]:
processName = dbutils.widgets.get('prm_processName')

nextSourceFileDateSql = f"""SELECT NVL(MAX(PROCESSED_FILE_TABLE_DATE)+1,'2023-01-01') as NEXT_SOURCE_FILE_DATE 
FROM pricing_analytics.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS 
WHERE PROCESS_NAME = '{processName}' and PROCESS_STATUS ='Completed'"""

nextSourceFileDateDF = spark.sql(nextSourceFileDateSql)
display(nextSourceFileDateDF)

In [0]:
from datetime import datetime

In [0]:
dailyPricingSourceBaseURL = 'https://retailpricing.blob.core.windows.net/'
dailyPricingSourceFolder = 'daily-pricing/'
dailyPricingSourceFileDate = datetime.strptime(str(nextSourceFileDateDF.select('NEXT_SOURCE_FILE_DATE').collect()[0]['NEXT_SOURCE_FILE_DATE']),'%Y-%m-%d').strftime('%m%d%Y')
dailyPricingSourceFileName = f"PW_MW_DR_{dailyPricingSourceFileDate}.csv"

dailyPricingSinkLayerName = 'bronze'
dailyPricingSinkStorageAccountName = 'adlsadataengdev'
dailyPricingSinkFolderName = 'daily-pricing'

In [0]:
import pandas as pds


In [0]:
dailyPricingSourceURL = dailyPricingSourceBaseURL + dailyPricingSourceFolder + dailyPricingSourceFileName

In [0]:
dailyPricingPandasDF = pds.read_csv(dailyPricingSourceURL)

In [0]:
dailyPricingSparkDF = spark.createDataFrame(dailyPricingPandasDF)

In [0]:
from pyspark.sql.functions import current_timestamp
dailyPricingSinkFolderPath = f"abfss://{dailyPricingSinkLayerName}@{dailyPricingSinkStorageAccountName}.dfs.core.windows.net/{dailyPricingSinkFolderName}"


(
    dailyPricingSparkDF
    .withColumn("source_file_load_date", current_timestamp())
    .write
    .mode("append")
    .option("header", "true")
    .csv(dailyPricingSinkFolderPath) 
)



In [0]:

processFileDate = nextSourceFileDateDF.select('NEXT_SOURCE_FILE_DATE').collect()[0]['NEXT_SOURCE_FILE_DATE']
processStatus = 'Completed'

processInsertSql = f""" INSERT INTO pricing_analytics.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS(PROCESS_NAME, PROCESSED_FILE_TABLE_DATE, PROCESS_STATUS) VALUES('{processName}', '{processFileDate}', '{processStatus}')"""

spark.sql(processInsertSql)